In [15]:
def preprocess_raw_image(this):

    # import gdal and couple other libraries #
    import os
    import gdal

    ## changing the GDAL env path to find proj package ##
    os.environ['PROJ_LIB'] = '/home/c3/.conda/envs/py-image/share/proj'
    os.environ['GDAL_DATA'] = '/home/c3/.conda/envs/py-image/share'

    ## a series checking for this C3 Class and see if it is ready for preprocessing #
    
    # test 1: status check #
    if(this.status != 'raw'):
        raise RuntimeError('The file is not ready to preprocess')

    def get_encoded_path(path):
        """
        Trim local fs url to absolute path
        e.g. from "file:///tmp/d.txt" to "/tmp/d.txt"
        """
        if path.startswith("file://"):
            return path[7:]
        return path

    def local_file(base_path):
        """
        Downloads file locally
        :param base_path: string to remote file system url
        :return: temp_path: string to local file system path
        """
        file = c3.LocalFileSystem.makeTmpFile().url
        c3.Client.copyFilesToLocalClient(srcUrlOrEncodedPath=base_path, localPath=file)
        temp_path = os.path.join(get_encoded_path(file), os.path.basename(base_path))
        return temp_path

    ## creating the tmp file in the space
    tmp_url = this.raw_image_file.url
    tmp_local = local_file(tmp_url)

    # test 2: raw image file path check #
    updated = c3.PlanetFile(**{'id':this.id})
    if(this.external_raw_path != None):
        try:
            ## print(updated.external_processed_path, updated.external_raw_path)
            updated.status = 'preprocessing'
            updated.external_processed_path = this.external_raw_path.replace('.tif', '-warp.tif')
            gdal_raw_fp = this.raw_image_file.contentLocation
            gdal_preprocessed_fp = gdal_raw_fp.replace('.tif', '-warp.tif')
            ## using the full path ##
            file_name = os.path.basename(updated.external_processed_path)
            folder_name = os.path.dirname(updated.external_processed_path)
            ## gdal.Warp(gdal_preprocessed_fp, gdal_raw_fp, dstSRS='EPSG:32616', xRes=3, yRes=3)
            ds = gdal.Open(tmp_local)
            options = gdal.WarpOptions(srcSRS='EPSG:3857', dstSRS = 'EPSG:32616', yRes=3, xRes=3)
            tmp_path = "/tmp/" + file_name
            gdal.Warp(srcDSOrSrcDSTab=ds, destNameOrDestDS=tmp_path, options=options)
            c3.Client.uploadLocalClientFiles(localPath=tmp_path, dstUrlOrEncodedPath=folder_name, spec={"peekForMetadata": True})
            updated.processed_image_file = c3.File(**{'url': updated.external_processed_path}).readMetadata()
            updated.merge()
        except Exception as e:
            updated.status = 'error'
            updated.merge()
            raise e

    return updated.external_processed_path

In [16]:
test_file = c3.PlanetFile.fetch({"filter": "id == 'test1_503-1275'"}).first()
test_file.download_raw_image()
test_file = c3.PlanetFile.get("test1_503-1275")
test_file

c3.PlanetFile(
 id='test1_503-1275',
 name='503-1275',
 meta=c3.Meta(
        tenantTagId=152,
        tenant='dev',
        tag='tc03d',
        created=datetime.datetime(2021, 12, 21, 15, 11, 3, tzinfo=datetime.timezone.utc),
        createdBy='zhang303@illinois.edu',
        updated=datetime.datetime(2022, 1, 28, 16, 42, 17, tzinfo=datetime.timezone.utc),
        updatedBy='zhang303@illinois.edu',
        timestamp=datetime.datetime(2022, 1, 28, 16, 42, 17, tzinfo=datetime.timezone.utc),
        fetchInclude='[]',
        fetchType='PlanetFile'),
 version=2,
 planet_collector=c3.PlanetCollector(id='test1'),
 raw_image_file=c3.AzureFile(
                  contentLength=110711150,
                  contentLocation='fs/dti/mpodolsky/yifang_guan/planet_collection/raw/test1/503-1275.tif',
                  contentType='application/json',
                  eTag='"0x8D9E27D25440E3A"',
                  lastModified=datetime.datetime(2022, 1, 28, 16, 42, 17, tzinfo=datetime.timezone.utc),
 

In [17]:
preprocess_raw_image(test_file)

'yifang_guan/planet_collection/raw/test1/503-1275-warp.tif'

In [18]:
test_file = c3.PlanetFile.get("test1_503-1275")
test_file

c3.PlanetFile(
 id='test1_503-1275',
 name='503-1275',
 meta=c3.Meta(
        tenantTagId=152,
        tenant='dev',
        tag='tc03d',
        created=datetime.datetime(2021, 12, 21, 15, 11, 3, tzinfo=datetime.timezone.utc),
        createdBy='zhang303@illinois.edu',
        updated=datetime.datetime(2022, 1, 28, 16, 42, 39, tzinfo=datetime.timezone.utc),
        updatedBy='zhang303@illinois.edu',
        timestamp=datetime.datetime(2022, 1, 28, 16, 42, 39, tzinfo=datetime.timezone.utc),
        fetchInclude='[]',
        fetchType='PlanetFile'),
 version=3,
 planet_collector=c3.PlanetCollector(id='test1'),
 raw_image_file=c3.AzureFile(
                  contentLength=110711150,
                  contentLocation='fs/dti/mpodolsky/yifang_guan/planet_collection/raw/test1/503-1275.tif',
                  contentType='application/json',
                  eTag='"0x8D9E27D25440E3A"',
                  lastModified=datetime.datetime(2022, 1, 28, 16, 42, 17, tzinfo=datetime.timezone.utc),
 

In [14]:
os.path.dirname(test_file.external_processed_path)

'yifang_guan/planet_collection/raw/test1'

In [1]:
import osgeo

In [5]:
root = '/'.join(osgeo.__path__[0].split('/')[0:-4])
os.environ['PROJ_LIB'] = root + '/share/proj'
os.environ['GDAL_DATA'] = root + '/share'

In [3]:
os.environ['PROJ_LIB']

'/home/c3/.conda/envs/py-image/share/proj'

In [6]:
os.environ['GDAL_DATA']

'/home/c3/.conda/envs/py-image/share'

In [7]:
from osgeo import gdal

In [8]:
gdal.Warp

<function osgeo.gdal.Warp(destNameOrDestDS, srcDSOrSrcDSTab, **kwargs)>